In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

data = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/00_data/data_with_lag/data_with_lag.csv")
#print(data.head())
#data brot:
df= data[data["Warengruppe_2"] == 1].copy()
data_wetter = pd.read_csv("/workspaces/bakery_sales_prediction/2_BaselineModel/02_Franz/weather_data_with_flags.csv")


df = df.drop(columns=["Warengruppe_1", "Warengruppe_2", "Warengruppe_3", "Warengruppe_4", "Warengruppe_5", "Warengruppe_6"])
df = df.drop(columns=["Datum"])
print(df)



          Umsatz  Temp_Very_Cold_current  Temp_Very_Cold_prev  \
1819  535.856285                       0                  0.0   
1820  546.780787                       0                  0.0   
1821  427.343259                       0                  0.0   
1822  454.859641                       0                  0.0   
1823  492.818804                       0                  0.0   
...          ...                     ...                  ...   
3633  677.415844                       0                  0.0   
3634  716.576825                       0                  0.0   
3635  721.822447                       0                  0.0   
3636  616.358562                       0                  0.0   
3637  586.081666                       0                  0.0   

      Temp_Very_Cold_next  Temp_Cold_current  Temp_Cold_prev  Temp_Cold_next  \
1819                  0.0                  0             0.0             0.0   
1820                  0.0                  0             0.

In [2]:
# 1. Schritt 1: Untersuche lineare Korrelation zwischen den einzelnen Features mit der Zielvariable
# mit Korrelationsmatrix
import pandas as pd

# Zeilen- und Spaltenanzeige in Pandas erweitern
pd.set_option("display.max_rows", None)  # Zeigt alle Zeilen
pd.set_option("display.max_columns", None)  # Zeigt alle Spalten

# Korrelationsmatrix erstellen
correlation_matrix = df.corr()

# Korrelation mit der Zielvariable
correlation_with_target = correlation_matrix["Umsatz"].sort_values(ascending=False)
print(correlation_with_target)




Umsatz                            1.000000
Sommer_prev                       0.565973
Sommer_current                    0.565432
Sommer_next                       0.563402
is_holiday_next                   0.413698
is_holiday_prev                   0.412688
Monat_7_prev                      0.407876
is_holiday_current                0.402799
Monat_7_current                   0.395344
Samstag_prev                      0.386913
Sonntag_current                   0.385942
Montag_next                       0.385861
Monat_8_next                      0.383135
Monat_7_next                      0.382623
Monat_8_current                   0.371096
Monat_8_prev                      0.356319
Temp_Warm_prev                    0.323011
Temp_Warm_current                 0.319574
Temp_Warm_next                    0.298007
wetter_sehr_schön_prev            0.268713
Number_of_ships_current           0.261063
Number_of_ships_scaled_current    0.261063
Ship_current                      0.254392
wetter_sehr

In [3]:
from sklearn.feature_selection import mutual_info_regression

# Nicht-lineare Zusammenhänge analysieren
X = df.drop("Umsatz", axis=1)
y = df["Umsatz"]
mi = mutual_info_regression(X, y)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print(mi_series)



VPI_next                          0.275993
VPI_prev                          0.268067
VPI_current                       0.264623
Sommer_next                       0.201213
Sommer_current                    0.200458
Sommer_prev                       0.196215
Werktag_current                   0.165373
Temp_Cold_current                 0.125101
Temp_Cold_prev                    0.114685
Temp_Cold_next                    0.106767
Monat_7_prev                      0.103952
Sonntag_current                   0.103903
Samstag_prev                      0.103004
Montag_next                       0.102103
is_holiday_next                   0.102013
Monat_7_current                   0.093496
is_holiday_current                0.091818
is_holiday_prev                   0.091731
Winter_prev                       0.086981
Monat_7_next                      0.083975
Winter_current                    0.083544
Monat_8_next                      0.081344
Winter_next                       0.078161
Monat_8_cur

In [4]:

import numpy as np

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Stark korrelierte Features (> 0.5) finden
high_corr = corr_matrix.abs() > 0.7
np.fill_diagonal(high_corr.values, False)  # Hauptdiagonale ignorieren
correlated_features = [column for column in high_corr.columns if any(high_corr[column])]
print(f"Stark korrelierte Features:", correlated_features)

Stark korrelierte Features: ['Temp_Cold_current', 'Temp_Cold_prev', 'Temp_Cold_next', 'Cloud_Cloudy_current', 'Cloud_Cloudy_prev', 'Cloud_Cloudy_next', 'Wind_Light_current', 'Wind_Light_prev', 'Wind_Light_next', 'Wind_Moderate_current', 'Wind_Moderate_prev', 'Wind_Moderate_next', 'Wind_Strong_current', 'Wind_Strong_prev', 'Wind_Strong_next', 'KielerWoche_current', 'KielerWoche_prev', 'KielerWoche_next', 'Montag_current', 'Montag_prev', 'Montag_next', 'Dienstag_current', 'Dienstag_prev', 'Dienstag_next', 'Mittwoch_current', 'Mittwoch_prev', 'Mittwoch_next', 'Donnerstag_current', 'Donnerstag_prev', 'Donnerstag_next', 'Freitag_current', 'Freitag_prev', 'Freitag_next', 'Samstag_current', 'Samstag_prev', 'Samstag_next', 'Sonntag_current', 'Sonntag_prev', 'Sonntag_next', 'VPI_current', 'VPI_prev', 'VPI_next', 'Number_of_ships_current', 'Number_of_ships_prev', 'Number_of_ships_next', 'Ship_current', 'Ship_prev', 'Ship_next', 'is_holiday_current', 'is_holiday_prev', 'is_holiday_next', 'Weihnac

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF für alle Features berechnen
X = df.drop("Umsatz", axis=1)
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)



/workspaces/bakery_sales_prediction/.venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                            Feature           VIF
0            Temp_Very_Cold_current           inf
1               Temp_Very_Cold_prev           inf
2               Temp_Very_Cold_next           inf
3                 Temp_Cold_current           inf
4                    Temp_Cold_prev  2.445569e+07
5                    Temp_Cold_next           inf
6                 Temp_Mild_current           inf
7                    Temp_Mild_prev           inf
8                    Temp_Mild_next           inf
9                 Temp_Warm_current           inf
10                   Temp_Warm_prev           inf
11                   Temp_Warm_next           inf
12                 Temp_Hot_current           inf
13                    Temp_Hot_prev           inf
14                    Temp_Hot_next           inf
15              Cloud_Clear_current           inf
16                 Cloud_Clear_prev           inf
17                 Cloud_Clear_next           inf
18      Cloud_Partly_Cloudy_current           inf


In [6]:
# suche miteinander korrelierrende features

In [7]:

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Schwellenwert für "stark korreliert"
threshold = 0.4

# Finde die Paare mit starker Korrelation
strong_corr_pairs = set()

# Iteriere über die obere Dreiecksmatrix der Korrelationsmatrix
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:  # Korrelation über dem Schwellenwert
            strong_corr_pairs.add((corr_matrix.columns[i], corr_matrix.columns[j]))

# Anzeigen der Paare starker Korrelationen
print("Stark korrelierte Feature-Paare:")
print(strong_corr_pairs)


Stark korrelierte Feature-Paare:
{('is_holiday_current', 'Umsatz'), ('Monat_2_next', 'Winter_next'), ('Monat_6_next', 'Sommer_prev'), ('Heimspiel_next', 'Freitag_current'), ('Sommer_current', 'Temp_Cold_current'), ('Monat_2_prev', 'Winter_prev'), ('Monat_7_prev', 'Sommer_next'), ('Sommer_prev', 'Temp_Cold_prev'), ('Winter_next', 'Temp_Mild_current'), ('Temp_Very_Cold_next', 'Temp_Very_Cold_prev'), ('Winter_prev', 'Temp_Cold_next'), ('Cloud_ok_next', 'Cloud_Partly_Cloudy_next'), ('Monat_10_next', 'Herbst_current'), ('Monat_4_current', 'Markt_next'), ('Sonntag_next', 'Samstag_current'), ('Weather_Moderate_next', 'Weather_Moderate_current'), ('Monat_4_next', 'Markt_prev'), ('Number_of_ships_scaled_next', 'Number_of_ships_next'), ('wetter_sehr_schön_current', 'Temp_Hot_next'), ('Monat_12_current', 'Weihnachtsmarkt_prev'), ('Monat_4_current', 'Frühling_current'), ('Monat_10_next', 'Herbst_next'), ('Weihnachtsmarkt_next', 'Weihnachtsmarkt_current'), ('zwischen_den_jahren_prev', 'Silvester_pr